In [1]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
from astropy.wcs import FITSFixedWarning
from astropy.wcs import WCS
import warnings
import tqdm
import os

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FITSFixedWarning)

In [2]:
def angular_distance(ra1, dec1, ra2, dec2):
    d1 = np.sin(np.radians(dec1)) * np.sin(np.radians(dec2))
    d2 = np.cos(np.radians(dec1)) * np.cos(np.radians(dec2)) * np.cos(np.radians(ra1 - ra2))
    return np.degrees(np.arccos(d1 + d2))

def max_radius(ra1, dec1, ra2, dec2):
    return np.nanmax(angular_distance(ra1, dec1, ra2, dec2))

In [3]:
ps1_folder = '/Users/zgl12/Modules/SynDiff/development/SkyCells/'
ps1_filename = 'Rings.Images.V3.fits'

# tess_file = '/Users/zgl12/Modules/SynDiff/development/TESS_FFI/TESS_FFI_Coordinates.csv'
skycell_csv = '/Users/zgl12/Modules/SynDiff/development/skycell_coordinates.csv'

tess_ffi_csv = '/Users/zgl12/Modules/SynDiff/development/TESS_FFI/TESS_FFI_Coordinates.csv'

path = ps1_folder


In [4]:
tess_ffi_df = pd.read_csv(tess_ffi_csv)

ra_center = tess_ffi_df['RA_center'].to_numpy()
dec_center = tess_ffi_df['DEC_center'].to_numpy()

In [5]:
if os.path.exists(skycell_csv):
    sky = True
    print('Skycell Coordinates already exist')
    new_df = pd.read_csv(skycell_csv)
else:

    hdul = fits.open(ps1_folder + ps1_filename)
    data = hdul[1].data
    header = hdul[0].header
    hdul.info()
    hdul.close()


    cols = data.columns.names
    df = pd.DataFrame(columns = cols)

    new_df = pd.DataFrame(columns = ['Name', 'RA', 'DEC', 'RA_Corner1', 'DEC_Corner1', 
                                     'RA_Corner2', 'DEC_Corner2', 'RA_Corner3', 'DEC_Corner3', 
                                     'RA_Corner4', 'DEC_Corner4'])

    for j in tqdm.tqdm(range(len(data) - 63500), desc='Processing PS1 Data'):
        temp = {}
        j += 63500
        for i in range(len(data[j])):
            temp[cols[i]] = data[j][i]

        temp.pop('POLYTERMS', None)

        wcs = WCS(fits.Header(temp))

        ra, dec = wcs.all_pix2world(temp['NX']//2, temp['NY']//2, 0)

        corners = np.array([[0, 0], [0, temp['NY']], [temp['NX'], temp['NY']], [temp['NX'], 0]])

        ras, decs = wcs.all_pix2world(corners[:,0], corners[:,1], 0)

        new_df.loc[len(new_df)] = [temp['NAME'], ra, dec, ras[0], decs[0], ras[1], decs[1], ras[2], decs[2], ras[3], decs[3]]
    
    new_df.to_csv(skycell_csv, index=False)

Skycell Coordinates already exist


In [18]:


for j in range(len(ra_center)):

    skycells = pd.DataFrame(columns = ['Name', 'Dist', 'RA', 'DEC', 'RA_Corner1', 'DEC_Corner1', 
                                     'RA_Corner2', 'DEC_Corner2', 'RA_Corner3', 'DEC_Corner3', 
                                     'RA_Corner4', 'DEC_Corner4'])

    decc = dec_center[j]
    rac = ra_center[j]

    ra_corners = np.array([tess_ffi_df.iloc[j][f'RA_corner{i}'] for i in range(1,5)])
    dec_corners = np.array([tess_ffi_df.iloc[j][f'DEC_corner{i}'] for i in range(1,5)])

    radius_var = max_radius(ra_center[j], dec_center[j], ra_corners, dec_corners) + np.sqrt(2) * 0.4

    ccd = int(tess_ffi_df.iloc[j]['CCD'] + 4*(tess_ffi_df.iloc[j]['Camera'] - 1))
    sector = int(tess_ffi_df.iloc[j]['Sector'])

    radius = angular_distance(rac, decc, new_df['RA'].to_numpy(), new_df['DEC'].to_numpy())

    mask = radius < radius_var

    temp_skycells = new_df[mask]

    names = temp_skycells['Name'].to_numpy()
    ra1 = temp_skycells['RA_Corner1'].to_numpy()
    dec1 = temp_skycells['DEC_Corner1'].to_numpy()
    ra2 = temp_skycells['RA_Corner2'].to_numpy()
    dec2 = temp_skycells['DEC_Corner2'].to_numpy()
    ra3 = temp_skycells['RA_Corner3'].to_numpy()
    dec3 = temp_skycells['DEC_Corner3'].to_numpy()
    ra4 = temp_skycells['RA_Corner4'].to_numpy()
    dec4 = temp_skycells['DEC_Corner4'].to_numpy()

    skycells['Name'] = names
    skycells['Dist'] = radius[mask]
    skycells['RA'] = temp_skycells['RA'].to_numpy()
    skycells['DEC'] = temp_skycells['DEC'].to_numpy()
    skycells['RA_Corner1'] = ra1
    skycells['DEC_Corner1'] = dec1
    skycells['RA_Corner2'] = ra2
    skycells['DEC_Corner2'] = dec2
    skycells['RA_Corner3'] = ra3
    skycells['DEC_Corner3'] = dec3
    skycells['RA_Corner4'] = ra4
    skycells['DEC_Corner4'] = dec4

    skycells = skycells.reset_index(drop=True)

    file = path + f"skycell_s{sector}_c{ccd}.csv"

    skycells.to_csv(file, index=False)

In [16]:
skycells

,Name,Dist,RA,DEC,RA_Corner1,DEC_Corner1,RA_Corner2,DEC_Corner2,RA_Corner3,DEC_Corner3,RA_Corner4,DEC_Corner4
0,skycell.0705.020,9.084073,320.797856,-31.005104,321.015703,-31.222743,321.015703,-30.787396,320.580009,-30.787396,320.580009,-31.222743
1,skycell.0705.030,8.842220,320.797856,-30.603090,321.015703,-30.820729,321.015703,-30.385382,320.580009,-30.385382,320.580009,-30.820729
2,skycell.0705.031,9.125140,320.395495,-30.603090,320.613342,-30.820729,320.613342,-30.385382,320.177648,-30.385382,320.177648,-30.820729
3,skycell.0705.040,8.612250,320.797856,-30.201076,321.015703,-30.418715,321.015703,-29.983368,320.580009,-29.983368,320.580009,-30.418715
4,skycell.0705.041,8.903601,320.395495,-30.201076,320.613342,-30.418715,320.613342,-29.983368,320.177648,-29.983368,320.177648,-30.418715
...,...,...,...,...,...,...,...,...,...,...,...,...
1562,skycell.0960.059,8.419833,333.073138,-17.799167,333.290985,-18.016458,333.290985,-17.581875,332.855290,-17.581875,332.855290,-18.016458
1563,skycell.0960.067,9.147195,333.877860,-17.397917,334.095707,-17.615208,334.095707,-17.180625,333.660013,-17.180625,333.660013,-17.615208
1564,skycell.0960.068,8.955570,333.475499,-17.397917,333.693346,-17.615208,333.693346,-17.180625,333.257652,-17.180625,333.257652,-17.615208
1565,skycell.0960.069,8.775741,333.073138,-17.397917,333.290985,-17.615208,333.290985,-17.180625,332.855290,-17.180625,332.855290,-17.615208


In [12]:
skycells

,Name,RA,DEC,RA_Corner1,DEC_Corner1,RA_Corner2,DEC_Corner2,RA_Corner3,DEC_Corner3,RA_Corner4,DEC_Corner4
0,skycell.0706.000,325.354832,-31.809132,325.572680,-32.026771,325.572680,-31.591424,325.136985,-31.591424,325.136985,-32.026771
1,skycell.0706.001,324.952471,-31.809132,325.170319,-32.026771,325.170319,-31.591424,324.734624,-31.591424,324.734624,-32.026771
2,skycell.0706.010,325.354832,-31.407118,325.572680,-31.624757,325.572680,-31.189410,325.136985,-31.189410,325.136985,-31.624757
3,skycell.0707.000,329.911778,-31.809132,330.129625,-32.026771,330.129625,-31.591424,329.693931,-31.591424,329.693931,-32.026771
4,skycell.0707.001,329.509417,-31.809132,329.727264,-32.026771,329.727264,-31.591424,329.291570,-31.591424,329.291570,-32.026771
...,...,...,...,...,...,...,...,...,...,...,...
356,skycell.0791.006,337.447305,-27.807813,337.664457,-28.025313,337.664457,-27.590243,337.230082,-27.590243,337.230082,-28.025313
357,skycell.0791.007,337.046263,-27.807813,337.263416,-28.025313,337.263416,-27.590243,336.829041,-27.590243,336.829041,-28.025313
358,skycell.0791.008,336.645221,-27.807813,336.862374,-28.025313,336.862374,-27.590243,336.427999,-27.590243,336.427999,-28.025313
359,skycell.0791.009,336.244180,-27.807813,336.461332,-28.025313,336.461332,-27.590243,336.026957,-27.590243,336.026957,-28.025313
